In [298]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from scipy.stats import ks_2samp, wasserstein_distance,energy_distance
import pickle
from scipy.spatial.distance import cdist, pdist
from scipy.optimize import minimize, LinearConstraint

In [299]:
# with open('compatibility_analysis.pkl', 'rb') as f:
#     saved_data = pickle.load(f)

# # Unpack
# embeddings_1 = saved_data['embeddings_1']
# embeddings_2 = saved_data['embeddings_2']
# cols1_h = saved_data['cols1_h']
# cols2_h = saved_data['cols2_h']
# compatibility_score = saved_data['compatibility_score']

# print("✅ Data loaded successfully.")
# print(f"↪️ Compatibility score: {compatibility_score:.2f}")
# print(f"↪️ Columns in Dataset 1: {len(cols1_h)}")
# print(f"↪️ Columns in Dataset 2: {len(cols2_h)}")

In [300]:
def convert_columns_to_numeric(df, file_label=""):
    print(f"\nProcessing {file_label}")
    print("Original dtypes:")
    print(df.dtypes)

    converted_cols = {}
    for col in df.columns:
        original_dtype = df[col].dtype
        if not pd.api.types.is_numeric_dtype(df[col]):
            df[col] = pd.to_numeric(df[col], errors='coerce')
            new_dtype = df[col].dtype
            if new_dtype != original_dtype:
                converted_cols[col] = (original_dtype, new_dtype)

    if converted_cols:
        print("\nColumns converted:")
        for col, (orig, new) in converted_cols.items():
            print(f" - {col}: {orig} -> {new}")
    else:
        print("\nNo columns were converted.")

    return df

# Replace these with your actual CSV file paths
file1_path = '../../Datasets/Ingestor_Datasets/Dataset7.csv'
file2_path = '../../Datasets/Ingestor_Datasets/Dataset8.csv'

df1 = pd.read_csv(file1_path)
df2 = pd.read_csv(file2_path)

df1 = convert_columns_to_numeric(df1, "File 1")
df2 = convert_columns_to_numeric(df2, "File 2")


Processing File 1
Original dtypes:
Ticker                         object
costOfRevenue                 float64
costofGoodsAndServicesSold    float64
ebitda                          int64
fiscalDateEnding               object
grossProfit                   float64
incomeBeforeTax                 int64
incomeTaxExpense                int64
dtype: object

Columns converted:
 - Ticker: object -> float64
 - fiscalDateEnding: object -> float64

Processing File 2
Original dtypes:
interestExpense                      float64
netIncome                              int64
netIncomeFromContinuingOperations    float64
operatingExpenses                    float64
operatingIncome                        int64
reportedCurrency                      object
researchAndDevelopment               float64
sellingGeneralAndAdministrative      float64
totalRevenue                         float64
dtype: object

Columns converted:
 - reportedCurrency: object -> float64


In [301]:
def run_ks_tests(df1, df2):
    ks_stat_matrix = pd.DataFrame(index=df1.columns, columns=df2.columns)
    p_value_matrix = pd.DataFrame(index=df1.columns, columns=df2.columns)

    for col1 in df1.columns:
        for col2 in df2.columns:
            # Drop missing values
            series1 = df1[col1].dropna()
            series2 = df2[col2].dropna()

            # Check if either series is empty after dropping NaN
            if len(series1) == 0 or len(series2) == 0:
                ks_stat_matrix.loc[col1, col2] = None
                p_value_matrix.loc[col1, col2] = None
                continue

            # Perform the KS test
            stat, p_value = ks_2samp(series1, series2)

            # Store the results in the matrices
            ks_stat_matrix.loc[col1, col2] = stat
            p_value_matrix.loc[col1, col2] = p_value

    return ks_stat_matrix, p_value_matrix


In [302]:
ks_stat_matrix, p_value_matrix = run_ks_tests(df1, df2)

In [303]:
def run_wasserstein_test(df1, df2):
    wasserstein_matrix = pd.DataFrame(index=df1.columns, columns=df2.columns)

    for col1 in df1.columns:
        for col2 in df2.columns:
            # Check if both columns contain numeric data
            if not pd.api.types.is_numeric_dtype(df1[col1]) or not pd.api.types.is_numeric_dtype(df2[col2]):
                wasserstein_matrix.loc[col1, col2] = None
                continue

            # Drop missing values
            series1 = df1[col1].dropna()
            series2 = df2[col2].dropna()

            # Check if either series is empty after dropping NaN
            if len(series1) == 0 or len(series2) == 0:
                wasserstein_matrix.loc[col1, col2] = None
                continue

            # Z-score normalization
            s1 = (series1 - series1.mean()) / series1.std()
            s2 = (series2 - series2.mean()) / series2.std()

            # Calculate Wasserstein distance
            dist = wasserstein_distance(s1, s2)
            wasserstein_matrix.loc[col1, col2] = dist

    return wasserstein_matrix


In [304]:
wasserstein_distance_matrix = run_wasserstein_test(df1, df2)

In [305]:
def calculate_psi(expected, actual, bins=2):
    expected = expected.dropna()
    actual = actual.dropna()

    if isinstance(bins, int):
        bin_edges = np.percentile(expected, np.linspace(0, 100, bins + 1))
        bin_edges = np.unique(bin_edges)  # Remove duplicates
        if len(bin_edges) < 2:
            return np.nan  # Cannot bin a constant column
    else:
        bin_edges = bins

    expected_bins = np.histogram(expected, bins=bin_edges)[0]
    actual_bins = np.histogram(actual, bins=bin_edges)[0]

    if expected_bins.sum() == 0 or actual_bins.sum() == 0:
        return np.nan  # Avoid divide by zero

    expected_dist = expected_bins / expected_bins.sum()
    actual_dist = actual_bins / actual_bins.sum()

    # Add small value to avoid log(0) or divide-by-zero
    epsilon = 1e-6
    psi = np.sum((expected_dist - actual_dist) * np.log((expected_dist + epsilon) / (actual_dist + epsilon)))

    return psi

def run_psi_tests(df1, df2, bins=4, epsilon=1e-4):
    psi_matrix = pd.DataFrame(index=df1.columns, columns=df2.columns)

    for col1 in df1.columns:
        for col2 in df2.columns:
            # Check if both columns contain numeric data
            if not pd.api.types.is_numeric_dtype(df1[col1]) or not pd.api.types.is_numeric_dtype(df2[col2]):
                psi_matrix.loc[col1, col2] = None
                continue

            # Drop missing values
            series1 = df1[col1].dropna()
            series2 = df2[col2].dropna()

            # Check if either series is empty after dropping NaN
            if len(series1) == 0 or len(series2) == 0:
                psi_matrix.loc[col1, col2] = None
                continue

            # Calculate PSI
            psi = calculate_psi(series1, series2, bins=bins)
            psi_matrix.loc[col1, col2] = psi if not np.isnan(psi) else epsilon

    return psi_matrix



In [306]:
psi_matrix = run_psi_tests(df1, df2, bins=4)

In [307]:
def run_energy_distance(df1, df2):
    energy_matrix = pd.DataFrame(index=df1.columns, columns=df2.columns)

    for col1 in df1.columns:
        for col2 in df2.columns:
            # Check if both columns contain numeric data
            if not pd.api.types.is_numeric_dtype(df1[col1]) or not pd.api.types.is_numeric_dtype(df2[col2]):
                energy_matrix.loc[col1, col2] = None
                continue

            # Drop missing values
            s1 = df1[col1].dropna()
            s2 = df2[col2].dropna()

            # Check if either series is empty after dropping NaN
            min_len = min(len(s1), len(s2))
            if min_len == 0:
                energy_matrix.loc[col1, col2] = None
                continue

            # Z-score normalization
            s1 = (s1 - s1.mean()) / s1.std()
            s2 = (s2 - s2.mean()) / s2.std()

            # Compute energy distance
            d_xy = cdist(s1.values.reshape(-1, 1), s2.values.reshape(-1, 1)).mean()
            d_xx = pdist(s1.values.reshape(-1, 1)).mean() if len(s1) > 1 else 0
            d_yy = pdist(s2.values.reshape(-1, 1)).mean() if len(s2) > 1 else 0

            energy_dist = 2 * d_xy - d_xx - d_yy
            energy_matrix.loc[col1, col2] = energy_dist

    return energy_matrix


In [308]:
energy_dist_matrix = run_energy_distance(df1, df2)

In [309]:
def normalize_matrix(matrix, higher_is_better=True, epsilon=1e-6):
    normalized_matrix = matrix.copy()

    if higher_is_better:
        # Normalize by dividing by the maximum value to scale between 0 and 1
        max_val = normalized_matrix.max().max()
        if max_val > 0:
            normalized_matrix = normalized_matrix / max_val
    else:
        # Invert the matrix to make smaller values larger
        normalized_matrix = 1 / (normalized_matrix + epsilon)

    return normalized_matrix

In [310]:
normalized_ks_stat_matrix = normalize_matrix(ks_stat_matrix, higher_is_better=False)
normalized_wasserstein_matrix = normalize_matrix(wasserstein_distance_matrix, higher_is_better=False)
normalized_psi_matrix = normalize_matrix(psi_matrix, higher_is_better=True)
normalized_energy_matrix = normalize_matrix(energy_dist_matrix, higher_is_better=False)

print("\nNormalized Matrices:")
print(ks_stat_matrix)
print(wasserstein_distance_matrix)
print(psi_matrix)
print(energy_dist_matrix)


Normalized Matrices:
                           interestExpense netIncome  \
Ticker                                None      None   
costOfRevenue                     0.717171  0.369697   
costofGoodsAndServicesSold        0.717171  0.369697   
ebitda                            0.700974  0.228648   
fiscalDateEnding                      None      None   
grossProfit                       0.768901  0.410894   
incomeBeforeTax                   0.640501  0.119704   
incomeTaxExpense                  0.410447  0.375252   

                           netIncomeFromContinuingOperations  \
Ticker                                                  None   
costOfRevenue                                       0.358615   
costofGoodsAndServicesSold                          0.358615   
ebitda                                              0.272975   
fiscalDateEnding                                        None   
grossProfit                                           0.3984   
incomeBeforeTax          

In [ ]:
# hyper-parameters
eps      = 0.05      # minimum weight
max_diff = 0.60      # maximum allowed difference between any two weights

# penalties — make these large enough that violating any rule is expensive
pen_sum   = 1e3      # penalty for sum(w) ≠ 1
pen_floor = 1e3      # penalty for w < eps
pen_diff  = 1e3      # penalty for |w[i] - w[j]| > max_diff

def combined_matrix(w):
    return (
        normalized_ks_stat_matrix   ** w[0]
      * normalized_wasserstein_matrix ** w[1]
      * normalized_psi_matrix       ** w[2]
      * normalized_energy_matrix    ** w[3]
      * (compatibility_score          ** w[4])
    ).clip(upper=1.0)  # just in case

def loss_penalty(w):
    # 1) match objective (we want to maximize diag → minimize negative mean diag)
    M = combined_matrix(w)
    diag = np.diag(M.values)
    obj  = -diag.mean()

    # 2) penalty for sum ≠ 1
    obj += pen_sum * (w.sum() - 1.0)**2

    # 3) penalty for floor violation
    obj += pen_floor * np.sum(np.maximum(0.0, eps - w)**2)

    # 4) penalty for any pairwise diff > max_diff
    diffs = [abs(w[i] - w[j]) for i in range(4) for j in range(i)]
    overs = [np.maximum(0.0, d - max_diff) for d in diffs]
    obj += pen_diff * np.sum([o*o for o in overs])

    return obj

x0    = np.ones(5) / 5        # now 5 weights
bnds  = [(eps,1.0)] * 5

res = minimize(
    loss_penalty,
    x0,
    method='L-BFGS-B',
    bounds=bnds,
    options={'ftol':1e-9}
)

w_opt = res.x
# finally, renormalize so that sum(w)=1 exactly
w_opt = np.clip(w_opt, eps, None)
w_opt /= w_opt.sum()

print("Penalized & renormalized weights:", w_opt)
print(" → sum =", w_opt.sum(), "  max diff =", w_opt.max()-w_opt.min())

# rebuild the final matrix
M_opt = combined_matrix(w_opt)

Penalized & renormalized weights: [0.2 0.2 0.2 0.2 0.2]
 → sum = 1.0   max diff = 0.0


In [312]:
# # Pack your weights however you like; e.g. as a dict for clarity
# weights = {
#     'ks':        w_opt[0],
#     'wasserstein': w_opt[1],
#     'psi':       w_opt[2],
#     'energy':    w_opt[3],
#     'compatibility': w_opt[4]
# }

# # Write them out
# with open('weights.pkl', 'wb') as f:
#     pickle.dump(weights, f)

# print("✅ Saved weights to weights.pkl")

In [313]:
# 1) Identify best‐matching columns from your M_opt:
i, j = np.unravel_index(np.nanargmax(M_opt.values), M_opt.shape)
best_col1 = M_opt.index[i]   # e.g. "strike"
best_col2 = M_opt.columns[j] # e.g. "open_interest"

# 2) Perform an inner merge on those columns
combined = pd.merge(
    df1,
    df2,
    left_on=best_col1,
    right_on=best_col2,
    how='inner'
)

# 3) Export
combined.to_csv('combined.csv', index=False)
print(f"✅ Wrote {len(combined)} rows to combined.csv (≤{len(df1)})")

✅ Wrote 91 rows to combined.csv (≤1487)
